# Busqueda de cuadricula

Nos permite espeficicar un rango de valores para que el clasificador ajuste automaticamente la mejor combinación de parametros de entrenamiento 

In [63]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from utilities import visualize_classifier

In [64]:
#Cargamos datos
input_file = 'data_random_forests.txt'
data = np.loadtxt(input_file, delimiter=',')
X,y = data[:,:-1], data[:,-1]

In [65]:
#clases
class_0 = np.array(X[y==0])
class_1 = np.array(X[y==1])
class_2 = np.array(X[y==2])

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state=5)

Definimos la cuadricula de parametros, por lo general se deja uno constante y se varia el otro. Luego al contrario, con el fin de encontrar el mejor n_estimators y max_depth

In [67]:
parameter_grid = [{'n_estimators':[100],'max_depth':[2,4,7,12,16]},
                  {'max_depth':[4],'n_estimators':[25,50,100,250]}]

In [68]:
#definimos las metricas que el clasificador deberia usar  para cada combinación 
metrics = ['precision_weighted','recall_weighted']

for metric in metrics:
    print("\n#### Buscando parametros óptimos para:", metric)
    
    classifier = GridSearchCV( ExtraTreesClassifier(random_state=0) , parameter_grid, cv=5, scoring=metric) 
    classifier.fit(X_train,y_train)
    
    
    params = classifier.cv_results_['params']
    scores_params = classifier.cv_results_['mean_test_score']
    print("\nPuntuación para cada cuadricula de parametros:")
    for i in range(len(params)):
        print(params[i], '-->', round(scores_params[i], 3) )
        
                
    print("\nMejores parametros", classifier.best_params_)
    
    y_pred = classifier.predict(X_test)
    print("\nReporte de rendimiento:\n")
    print(classification_report(y_test, y_pred))


#### Buscando parametros óptimos para: precision_weighted

Puntuación para cada cuadricula de parametros:
{'max_depth': 2, 'n_estimators': 100} --> 0.85
{'max_depth': 4, 'n_estimators': 100} --> 0.841
{'max_depth': 7, 'n_estimators': 100} --> 0.844
{'max_depth': 12, 'n_estimators': 100} --> 0.832
{'max_depth': 16, 'n_estimators': 100} --> 0.816
{'max_depth': 4, 'n_estimators': 25} --> 0.846
{'max_depth': 4, 'n_estimators': 50} --> 0.84
{'max_depth': 4, 'n_estimators': 100} --> 0.841
{'max_depth': 4, 'n_estimators': 250} --> 0.845

Mejores parametros {'max_depth': 2, 'n_estimators': 100}

Reporte de rendimiento:

              precision    recall  f1-score   support

         0.0       0.94      0.81      0.87        79
         1.0       0.81      0.86      0.83        70
         2.0       0.83      0.91      0.87        76

    accuracy                           0.86       225
   macro avg       0.86      0.86      0.86       225
weighted avg       0.86      0.86      0.86       225